In [ ]:
key="sk-3FVL0GPHPecirvTIoOq5b2SEIBd1V-3O7-iL7d9vb8T3BlbkFJbZBD_8jJc-PNrbj8qT5Q82RiPRT-oFZQblFkFZ4yMA"

In [ ]:
!pip install openai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os
import openai
from openai import OpenAI

In [ ]:
openai.api_key=key
client=OpenAI(api_key=key)

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 89.7 MB/s eta 0:00:00


In [ ]:
import csv
import pdfplumber
import requests
from bs4 import BeautifulSoup
import openai  # Make sure to have openai installed and configured
import re


def extract_text_from_pdfs(pdf_paths):
    all_text = ""
    for pdf_path in pdf_paths:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                all_text += page.extract_text() + "\n"
    return all_text

def extract_text_from_urls(urls):
    all_text = ""
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        all_text += soup.get_text() + "\n"
    return all_text

def generate_one_question(chunk, max_tokens=150):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert on cybersecurity."},
            {"role": "user", "content": f"Generate a multiple-choice question with four options and indicate the correct answer based on the following text:\n\n{chunk}"}
        ],
        max_tokens=max_tokens,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def save_question_to_csv(questions, filename='questions.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = ['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer']
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()

        for question in questions:
            try:
                # Extract the question text
                question_match = re.match(r'Question \d+: (.+)', question)
                question_text = question_match.group(1).strip() if question_match else None

                # Extract the options
                options_match = re.findall(r'([A-D])\)\s*(.+)', question)
                options = [opt[1].strip() for opt in options_match]

                # Extract the correct answer
                correct_answer_match = re.search(r'Correct Answer:\s*([A-D])\)\s*(.+)', question)
                correct_option_letter = correct_answer_match.group(1).strip() if correct_answer_match else None
                correct_option_text = correct_answer_match.group(2).strip() if correct_answer_match else None

                if len(options) != 4:
                    print(f"Unexpected number of options: {options}. Skipping...")
                    continue

                # Ensure correct option letter matches the correct option text
                if correct_option_letter and correct_option_text:
                    correct_option_full = f"{correct_option_letter}) {correct_option_text}"
                    if correct_option_full not in options:
                        print(f"Correct answer {correct_option_full} not found in options: {options}. Skipping...")
                        continue

                # Map options to A, B, C, D
                option_a, option_b, option_c, option_d = options

                # Write the row to the CSV file
                writer.writerow({
                    'Question': question_text,
                    'Option A': option_a,
                    'Option B': option_b,
                    'Option C': option_c,
                    'Option D': option_d,
                    'Correct Answer': correct_option_letter
                })
            except (IndexError, ValueError) as e:
                print(f"Error processing question: {question}. Error: {e}")






In [ ]:
pdf_paths = [
    '/content/ATTACK_Design_and_Philosophy_March_2020.pdf',
    '/content/CELEX_32016R0679_EN_TXT.pdf',
    '/content/diamond.pdf',
    '/content/nist.sp.800-150.pdf'
]
urls = [
    'https://oasis-open.github.io/cti-documentation/',
    'https://capec.mitre.org'
]

pdf_text = extract_text_from_pdfs(pdf_paths)
url_text = extract_text_from_urls(urls)
combined_text = pdf_text + url_text
text_chunks = [combined_text[i:i + 2000] for i in range(0, len(combined_text), 2000)]  # Adjust chunk size as needed


In [ ]:
import time

# Determine chunk size and number of questions
chunk_size = 2000  # Adjust chunk size as needed
text_length = len(combined_text)
num_questions = 2500  # Define how many questions you want to generate

# Create chunks of text
text_chunks = [combined_text[i:i + chunk_size] for i in range(0, text_length, chunk_size)]

# Generate questions
questions = []
for i in range(num_questions):
    start_idx = (i * chunk_size) % text_length

    # Wrap around text if needed
    if start_idx + chunk_size <= text_length:
        chunk = combined_text[start_idx: start_idx + chunk_size]
    else:
        chunk = combined_text[start_idx:] + combined_text[:(start_idx + chunk_size) % text_length]

    print(f"Generating question {i + 1}/{num_questions}")
    response = generate_one_question(chunk) # Store the entire response
    questions.append(response)

    # Adding delay to respect rate limits
    time.sleep(1)

# Print the generated questions to inspect their structure
for i, question in enumerate(questions):
    print(f"Question {i + 1}: {question}")
    print("-" * 20) # Add a separator for clarity

Streaming output truncated to the last 5000 lines.
Correct Answer: C) The data subject has the right to transmit personal data to another controller without hindrance.
--------------------
Question 1948: Which of the following statements is true regarding the right to object in the context of data processing for direct marketing purposes?

A) The data subject cannot object to processing for direct marketing purposes.
B) The data subject can object to processing for direct marketing purposes only once.
C) The data subject shall have the right to object at any time to processing for direct marketing purposes.
D) The data subject can only object to processing for direct marketing purposes if it is related to information society services.

Correct answer: C) The data subject shall have the right to object at any time to processing for direct marketing purposes.
--------------------
Question 1949: Which of the following scenarios allows for automated individual decision-making, including pr

In [ ]:
print(questions)

[]


In [ ]:
def parse_question(question_text):
    # Ensure the question is formatted consistently
    if 'Correct Answer' in question_text:
        parts = question_text.split('\n')
        question = parts[0].strip()
        correct_answer = ""
        options = [part.strip() for part in parts[1:] if part.strip()]

        if len(options) >= 4:
            # Extract options A, B, C, D
            option_a = options[0] if len(options) > 0 else ""
            option_b = options[1] if len(options) > 1 else ""
            option_c = options[2] if len(options) > 2 else ""
            option_d = options[3] if len(options) > 3 else ""

            # Find the correct answer
            for option in options:
                if "Correct Answer" in option:
                    correct_answer = option.split(':')[-1].strip()
                    break

            return [question, option_a, option_b, option_c, option_d, correct_answer]
        else:
            return [question_text, "", "", "", "", ""]
    else:
        return [question_text, "", "", "", "", ""]

def save_questions_to_csv(questions, filename='questions.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer'])
        for question in questions:
            parsed_question = parse_question(question)
            writer.writerow(parsed_question)


In [ ]:

# Save the generated questions to a CSV file
save_questions_to_csv(questions, 'questions.csv')

print("Questions saved to 'questions.csv'")

Questions saved to 'questions.csv'


In [ ]:
questions = []
chunk_size = 2000
num_questions = 2500

text_length = len(combined_text)
for i in range(num_questions):
    start_idx = (i * chunk_size) % text_length
    chunk = combined_text[start_idx: start_idx + chunk_size]
    print(f"Generating question {i + 1}/{num_questions}")
    question = generate_one_question(chunk)
    questions.append(question)
    time.sleep(1)  # Adding delay to respect rate limits

save_questions_to_csv(questions)
print("Questions generated and saved to CSV.")

NameError: name 'combined_text' is not defined